In [43]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
import sys

import backtrader as bt

In [36]:
class TestSizer(bt.Sizer):
    params = dict(stake=1)

    def _getsizing(self, comminfo, cash, data, isbuy):
        dt, i = self.strategy.datetime.date(), data._id
        s = self.p.stake * (1 + (not isbuy))
        print('{} Data {} Type {} Sizing to {}'.format(
            dt, data._name, ('buy' * isbuy) or 'sell', s))

        return s

In [53]:
class St(bt.Strategy):
    params = dict(
        enter=[1, 3, 4],  # data ids are 1 based
        hold=[7, 10, 15],  # data ids are 1 based
        usebracket=True,
        rawbracket=True,
        pentry=0.015,
        plimits=0.03,
        pstop=0.02,
        valid=10,
    )

    def notify_order(self, order):
        if order.status == order.Submitted:
            return

        dt, dn = self.datetime.date(), order.data._name
        print('{} {} Order {} Status {}'.format(
            dt, dn, order.ref, order.getstatusname())
        )

        whichord = ['main', 'stop', 'limit', 'close']
        if not order.alive():  # not alive - nullify
            dorders = self.o[order.data]
            idx = dorders.index(order)
            dorders[idx] = None
            print('-- No longer alive {} Ref'.format(whichord[idx]))

            if all(x is None for x in dorders):
                dorders[:] = []  # empty list - New orders allowed

    def __init__(self):
        self.o = dict()  # orders per data (main, stop, limit, manual-close)
        self.holding = dict()  # holding periods per data

    def next(self):
        for i, d in enumerate(self.datas):
            dt, dn = self.datetime.date(), d._name
            pos = self.getposition(d).size
            print('{} {} Position {} Price{}'.format(dt, dn, pos, self.getposition(d).price))

            if not pos and not self.o.get(d, None):  # no market / no orders
                if dt.weekday() == self.p.enter[i]:
                    if not self.p.usebracket:
                        self.o[d] = [self.buy(data=d)]
                        print('{} {} Buy {}'.format(dt, dn, self.o[d][0].ref))

                    else:
                        p = d.close[0] * (1.0 - self.p.pentry)
                        pstp = p * (1.0 - self.p.pstop)
                        plmt = p * (1.0 + self.p.plimits)
                        valid = datetime.timedelta(self.p.valid)

                        if self.p.rawbracket:
                            o1 = self.buy(data=d, exectype=bt.Order.Limit,
                                          price=p, valid=valid, transmit=False)

                            o2 = self.sell(data=d, exectype=bt.Order.Stop,
                                           price=pstp, size=o1.size,
                                           transmit=False, parent=o1)

                            o3 = self.sell(data=d, exectype=bt.Order.Limit,
                                           price=plmt, size=o1.size,
                                           transmit=True, parent=o1)

                            self.o[d] = [o1, o2, o3]

                        else:
                            self.o[d] = self.buy_bracket(
                                data=d, price=p, stopprice=pstp,
                                limitprice=plmt, oargs=dict(valid=valid))

                        print('{} {} Main {} Stp {} Lmt {}'.format(
                            dt, dn, *(x.ref for x in self.o[d])))

                    self.holding[d] = 0

            elif pos:  # exiting can also happen after a number of days
                self.holding[d] += 1
                if self.holding[d] >= self.p.hold[i]:
                    o = self.close(data=d)
                    self.o[d].append(o)  # manual order to list of orders
                    print('{} {} Manual Close {}'.format(dt, dn, o.ref))
                    if self.p.usebracket:
                        self.cancel(self.o[d][1])  # cancel stop side
                        print('{} {} Cancel {}'.format(dt, dn, self.o[d][1]))

In [54]:
if __name__ == '__main__':
    
    orig_stdout = sys.stdout
    f = open('/home/aviral/Desktop/Algo_trading_iitk/Multidata_sample.txt' , 'w')
    sys.stdout = f
    
    cerebro = bt.Cerebro()
    folder_path = '/home/aviral/backtrader-master/datas/'
    data0_path = folder_path + 'orcl-1995-2014.txt'
    data1_path = folder_path + 'nvda-1999-2014.txt'
    data2_path = folder_path + 'yhoo-1996-2014.txt'

    fdate = datetime.datetime(2001,1,1)
    todate = datetime.datetime(2007, 1, 1)
    # Data feed
    data0 = bt.feeds.YahooFinanceCSVData(dataname=data0_path, fromdate=fdate, todate=todate)
    cerebro.adddata(data0, name='d0')

    data1 = bt.feeds.YahooFinanceCSVData(dataname=data1_path, fromdate=fdate, todate=todate)
    data1.plotinfo.plotmaster = data0
    cerebro.adddata(data1, name='d1')

    data2 = bt.feeds.YahooFinanceCSVData(dataname=data2_path, fromdate=fdate, todate=todate)
    data2.plotinfo.plotmaster = data0
    cerebro.adddata(data2, name='d2')

    cerebro.broker.setcash(100000)
    cerebro.broker.setcommission(commission=0.001)

    cerebro.addsizer(TestSizer, stake=5)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='annual_return')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    cerebro.addstrategy(St)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    strats = cerebro.run()
    strat = strats[0]
    
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    sys.stdout = orig_stdout
    f.close()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Final Portfolio Value: 99895.66


In [49]:
print('Sharpe Ratio:', strat.analyzers.sharpe.get_analysis())
print("\nDraw Down \n")
for x in strat.analyzers.drawdown.get_analysis():
    print(x, ": ", strat.analyzers.drawdown.get_analysis()[x])
    
print('\nAnnual Return: \n')
for x in strat.analyzers.annual_return.get_analysis():
    print(x, ": ", strat.analyzers.annual_return.get_analysis()[x])

Sharpe Ratio: OrderedDict([('sharperatio', -242.48146441109623)])

Draw Down 

len :  1474
drawdown :  0.027477491615436823
moneydown :  27.47930833294231
max :  AutoOrderedDict([('len', 1474), ('drawdown', 0.027703656699704364), ('moneydown', 27.70548837046954)])

Annual Return: 

2001 :  -4.7198502415968235e-05
2002 :  -2.161014467583211e-05
2003 :  -6.982167240598969e-06
2004 :  -9.48592995175801e-06
2005 :  -2.185669077503505e-06
2006 :  -0.00012122710791917513
